In [1]:
!pip install konlpy
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 32.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import urllib.request
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model



In [6]:


# 모델을 불러올 경로
model_path = '/content/drive/MyDrive/smishing_model.h5'

# 모델 불러오기
loaded_model = load_model(model_path)

okt = Okt()

# Doc2Vec 모델 불러오기
doc2vec_model = Doc2Vec.load("/content/drive/MyDrive/doc2vec_model")

def classify_smishing(input_text):
    # 형태소 분석기 사용 (Okt)
    tokens = okt.morphs(input_text)  # 토큰화


 # 정수 인덱스로 변환
    encoded = [[doc2vec_model.wv.key_to_index[word] for word in tokens if word in doc2vec_model.wv.key_to_index]]
    pad_tokens = pad_sequences(encoded, maxlen=100, padding='post')

    # 모델 예측
    smishing_score = loaded_model.predict(pad_tokens.astype('float32'))[0][0]

    # 스미싱 여부 판단 결과 출력
    if smishing_score > 0.5:
        print("{:.2f}% 확률로 스미싱입니다.".format(smishing_score * 100))
    else:
        print("{:.2f}% 확률로 일반 메시지입니다.".format((1 - smishing_score) * 100))

# 사용자로부터 텍스트 입력 받기
user_input_text = input("텍스트를 입력하세요: ")

# 스미싱 여부 판단 수행
classify_smishing(user_input_text)


텍스트를 입력하세요: 진짜 너무 졸리다....
1/1 [==============================] - 1s 837ms/step
99.86% 확률로 일반 메시지입니다.
